# Connect to DuckDB

This cell establishes a connection to the DuckDB database file `sakila.duckdb` so we can query the data. The connection uses a relative path from the `notebooks/` folder.


In [16]:
import duckdb
import pandas as pd

# Connect to DuckDB (relative path from notebooks folder)
con = duckdb.connect("../data/sakila.duckdb")




### Preview Staging Tables

This cell shows all tables in the `staging` schema and previews the first 5 rows of the `actor` table.


In [ ]:
tables = con.execute("SHOW TABLES").fetchall()
staging_tables = con.execute(
    "SELECT table_name FROM information_schema.tables WHERE table_schema='staging'"
).fetchall()
print("Tables in staging schema:", staging_tables)

### first 5 rows
This code shows first 5 rows of the table actor where we can see 

In [ ]:

sample_data = con.execute("SELECT * FROM staging.actor LIMIT 5").fetchdf()
sample_data


### 1a Identify Movies Longer Than 3 Hours

Objective: Find all movies with a duration greater than 180 minutes and display their title and length.

In [ ]:
long_movies = con.execute("""
    SELECT title, length 
    FROM staging.film 
    WHERE length > 180
""").df()
long_movies


### 1b Movies Containing the Word “Love” in the Title

Objective: List movies whose titles contain “love” along with their rating, length, and description.

In [ ]:
love_movies = con.execute("""
    SELECT title, rating, length, description
    FROM staging.film
    WHERE title ILIKE '%love%'
""").df()

love_movies

### 1c) Descriptive Statistics of Movie Length

Objective: Compute summary statistics for the length column, including the shortest, longest, average, and median movie lengths.

In [ ]:
length_stats = con.execute("""
    SELECT
        MIN(length) AS shortest,
        AVG(length) AS average,
        MEDIAN(length) AS median,
        MAX(length) AS longest
    FROM staging.film
""").df()

length_stats


### 1d) Top 10 Most Expensive Movies to Rent Per Day

Objective: Determine the movies with the highest rental rate per day.

In [ ]:
expensive_movies = con.execute("""
    SELECT title, rental_rate, rental_duration, (rental_rate / rental_duration) AS cost_per_day
    FROM staging.film
    ORDER By cost_per_day DESC
    LIMIT 10
""").df()
expensive_movies

### 1e Top 10 Actors by Number of Movies

Objective: Identify the actors who have appeared in the most films.

In [ ]:
top_actors = con.execute("""
    SELECT a.first_name, a.last_name, COUNT(fa.film_id) AS movie_count
    FROM staging.actor a
    JOIN staging.film_actor fa ON a.actor_id = fa.actor_id
    GROUP BY a.actor_id, a.first_name, a.last_name
    ORDER BY movie_count DESC
    LIMIT 10
""").df()

top_actors
                         
                         
                         
               



### **Custom Question 1: Which film has the highest replacement cost?**

**Objective:**  
Identify the film with the highest *replacement cost*, which indicates the most expensive movie to replace in the inventory.

**Explanation:**  
The `replacement_cost` column represents how much it costs the store to replace a lost or damaged film. Finding the highest value helps the business understand which films are most valuable.

**Query:**


In [ ]:
highest_replacement = con.execute("""
    SELECT title, replacement_cost
    FROM staging.film
    ORDER BY replacement_cost DESC
    LIMIT 1
""").df()

highest_replacement

### **Custom Question 2: How many customers do we have in each city?**

**Objective:**  
Understand how the customer base is distributed across cities.

**Explanation:**  
This helps reveal where most of the customers live. Cities with higher customer counts may need more marketing focus, promotions, or store attention.

**Query:**


In [ ]:
customers_by_city = con.execute("""
    SELECT ci.city, COUNT(c.customer_id) AS customer_count
    FROM staging.customer c
    JOIN staging.address a ON c.address_id = a.address_id
    JOIN staging.city ci ON a.city_id = ci.city_id
    GROUP BY ci.city
    ORDER BY customer_count DESC
""").df()

customers_by_city


### **Custom Question 3: What is the average rental duration for each film rating?**

**Objective:**  
Compare the average number of days customers keep movies with different MPAA ratings (G, PG, PG-13, R, NC-17).

**Explanation:**  
This helps reveal whether certain types of movies tend to be rented for longer periods (for example, family movies vs. adult-rated movies).

**Query:**


In [ ]:
avg_rental_duration = con.execute("""
    SELECT rating, AVG(rental_duration) AS avg_duration
    FROM staging.film
    GROUP BY rating
    ORDER BY avg_duration DESC
""").df()

avg_rental_duration


### **Task 2(a): Top 5 Customers by Total Spend**

**Objective:**  
Identify the five customers who have spent the most money in total. This helps the business recognize its top customers and potentially reward them with discounts or special offers.

**Explanation:**  
We aggregate all payments made by each customer, calculate their total spend, then visualize the top 5 customers in a bar chart.


In [ ]:
import matplotlib.pyplot as plt

top_customers = con.execute("""
    SELECT c.first_name || ' ' || c.last_name AS customer_name,
           SUM(p.amount) AS total_spend
    FROM staging.payment p
    JOIN staging.customer c ON p.customer_id = c.customer_id
    GROUP BY customer_name
    ORDER BY total_spend DESC
    LIMIT 5
""").df()

# Plot
plt.bar(top_customers['customer_name'], top_customers['total_spend'])
plt.xlabel("Customer")
plt.ylabel("Total Spend ($)")
plt.title("Top 5 Customers by Total Spend")
plt.xticks(rotation=45)
plt.show()


### **Task 2(b): Total Revenue by Film Category**

**Objective:**  
Determine how much revenue each movie category generates.

**Explanation:**  
By linking payment, rental, inventory, film, and category tables, we can calculate the total revenue for each film category.  
This bar chart shows which categories are most profitable, helping the business understand where demand is highest.


In [ ]:
category_revenue = con.execute("""
    SELECT c.name AS category,
           SUM(p.amount) AS total_revenue
    FROM staging.payment p
    JOIN staging.rental r ON p.rental_id = r.rental_id
    JOIN staging.inventory i ON r.inventory_id = i.inventory_id
    JOIN staging.film f ON i.film_id = f.film_id
    JOIN staging.film_category fc ON f.film_id = fc.film_id
    JOIN staging.category c ON fc.category_id = c.category_id
    GROUP BY c.name
    ORDER BY total_revenue DESC
""").df()

plt.bar(category_revenue['category'], category_revenue['total_revenue'])
plt.xlabel("Category")
plt.ylabel("Total Revenue ($)")
plt.title("Revenue per Film Category")
plt.xticks(rotation=90)
plt.show()
